![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Callysto’s Weekly Data Visualization

## Zero-emission Charging Stations in North America

### Recommended Grade levels: 8-12
<br>

### Instructions
#### “Run” the cells to see the graphs
Click “Cell” and select “Run All”.<br> This will import the data and run all the code, so you can see this week's data visualization. Scroll to the top after you’ve run the cells.<br> 

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don’t need to do any coding to view the visualizations**.
The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer? 
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

# Question

Have you ever wondered about how far cars with newer fuel types have to travel to charge?

Zero-emission vehicles are becoming increasingly popular and as the world becomes more environmentally conscious, the government is providing more funding to support these vehicles.

With a data source containing information on zero-emission vehicles, we can explore questions related to charging stations across North America.

### Goal

Our goal is to use zero-emission vehicle charging stations data to get a better understanding the layout of these stations across North America. 

We will use longitude and latitude points of these stations and plot them on a geographical map using plotly (a popular python library used for creating plots). We can also see the distance someone would have to travel from a specific address to reach one of these charging stations. 

We will also use a bar chart to compare the distance needed to travel for each fuel type.

The fuel types we will explore are:
- Biodiesel
- CNG
- Electric
- Ethanol
- Hydrogen
- LNG
- Propane
- Renewable Diesel

With these visualizations, we will get a better understanding on where different charging stations are located.

# Gather

### Code:
The code below will import the Python programming libraries we need to gather and organize the data to answer our question.

In [1]:
#import libraries
import pandas as pd
import plotly.express as px
from geopy.geocoders import Nominatim
import geopy.distance as dist
import folium

### Data:

The data source we will be using is provided by Transport Canada and is located on the government of Canada website: https://tc.canada.ca/en/road-transportation/innovative-technologies/zero-emission-vehicles/zero-emission-vehicle-charging-stations#/find/nearest?country=CA

We downloaded the data from the website as Comma Seperated Values (csv) and load it directly to Jupyter Notebook.

In [3]:
#import data from csv
df = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/zero-emission-charging-stations/zero-emission-charging-stations.csv',dtype='unicode')

#keep only the columns we are interested in
df = df[['Fuel Type Code','Station Name','Street Address','City','State','ZIP','Latitude','Longitude']]
df

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

# Organize

The following code snippets will help arrange and clean the data so that we can do analysis on it. This is a quality control step for our data and involves examining the data to detect anything odd with the data (e.g. structure, missing values), fixing the oddities, and checking if the fixes worked. 

The first thing we do is get the full name of the fuel type rather than having just the abbreviation. This step will help make the map we will generate easier to read as it may be difficult to understand what the abbreviation stands for. For example, it's not implied that 'LPG' means 'Propane'


In [ ]:
type_dict = {'BD':'Biodiesel','CNG':'CNG','ELEC':'Electric','E85':'Ethanol','HY':'Hydrogen','LNG':'LNG','LPG':'Propane','RD':'Renewable Diesel'}
df['Fuel Type'] = df['Fuel Type Code'].map(lambda x: type_dict[x])
df['Latitude'] = df['Latitude'].map(lambda x: float(x))
df['Longitude'] = df['Longitude'].map(lambda x: float(x))
df

By provided a starting address, we can use the geopy library to get the longitude and latitude of the address as well as use a distance calculation to find the closest stations to are starting location. You can change the starting location to any address if you are curious to what's closest to it. Currently the starting address is the Cybera office in Calgary.

In [ ]:
geolocator = Nominatim(user_agent="Callysto-EV-Notebook")
start = "3512 33 St NW Calgary, AB" #you can change this line of code to any address you would like

location = geolocator.geocode(start)
address = location.latitude,location.longitude,start

distances = {'Biodiesel':100000,'CNG':100000,'Electric':100000,'Ethanol':100000,'Hydrogen':100000,'LNG':100000,'Propane':100000,'Renewable Diesel':100000}
index = {'Biodiesel':100000,'CNG':100000,'Electric':100000,'Ethanol':100000,'Hydrogen':100000,'LNG':100000,'Propane':100000,'Renewable Diesel':100000}
all_distances = []

for i,row in df.iterrows():

    station = row['Latitude'],row['Longitude']
    distance = dist.geodesic(address[:2],station[:2]).km
    all_distances.append(distance)
    distances[row['Fuel Type']] = min(distances[row['Fuel Type']],distance)
    
    if distances[row['Fuel Type']] == distance:
        index[row['Fuel Type']] = i

df['Distance from start'] = all_distances

Another thing we want to do, which will be used for later visualizations, is to create a list of addresses of the nearest fuel type to the starting location we specified previously. This list will contain longitude and latitude information as well as the address of the station with its fuel type.

In [ ]:
addresses = []
addresses.append(address)
for n,i in index.items():
    tup = df.iloc[i]['Latitude'],df.iloc[i]['Longitude'],df.iloc[i]['Station Name'] + ' - ' + df.iloc[i]['Street Address'] + ', '+ df.iloc[i]['City'] + ', ' + df.iloc[i]['State'] + ' - ' + df.iloc[i]['Fuel Type']
    addresses.append(tup)
    
addresses

Finally, we will also create some dictionaries that will be used to make the map a little more prettier and add some life to it.

In [ ]:
colors = {start:'black','Biodiesel':'orange','CNG':'blue','Electric':'green','Ethanol':'red','Hydrogen':'darkgreen','LNG':'cadetblue','Propane':'darkpurple','Renewable Diesel':'beige'}
icons = {start:'home','Biodiesel':'gas-pump','CNG':'gas-pump','Electric':'gas-pump','Ethanol':'gas-pump','Hydrogen':'gas-pump','LNG':'gas-pump','Propane':'gas-pump','Renewable Diesel':'gas-pump'}

# Explore (1)

The code below will be used to help us look for evidence to answer our question on where charging stations are located. This can involve looking at data in table format, applying math and statistics, and creating different types of visualizations to represent our data.

We can plot these the stations in North America using there longitude and latitude values in a scatter_mapbox plot, which is just a geographical map. This may give us an easy way to characterize these pitches.

In [ ]:
fig = px.scatter_mapbox(df,lat='Latitude',lon='Longitude',color='Fuel Type',labels='Fuel Type Code',zoom=2,height=600,hover_data={'Station Name','Street Address'})
fig.update_layout(mapbox_style='open-street-map')
fig.show()

# Interpret

From this visualization, we can gather a lot of valuable information. We gain a better understanding of where each fuel type is located in North America and where you can find more stations supporting that type. For example we see that Biodiesel is more readily found in North-East USA, near Minnisota and Kansas City. We are also able to gather that the most readily available fuel types are Propare and Electric. Renewable Diesel only seems to be found around Los Angeles and is the "rarest" type of charging station.

# Explore (2)

Another thing we wanted to explore is how far is the closest charging station, by fuel type, from our starting location

Let's try plotting another map to see the distance we would have to travel. This time we will use a folium map, which is better in this scenario as it allows us to differentiate the points a little more.

In [ ]:
m = folium.Map()
sw = min(addresses)
ne = max(addresses)
m.fit_bounds([sw,ne])

for x in addresses:
    location = x[2]
    folium.Marker(x[:2],popup=folium.Popup(location,min_width=150,max_width=150),icon=folium.Icon(color=colors[location.split('- ')[-1]],icon=icons[location.split('- ')[-1]],prefix='fa')).add_to(m)

m

# Interpret

With this visualization, we are able to see how far each fueling station is from our starting location. Depending on where you put your starting location as, the distance away from from each fuel type will be different. For example, if you decided to have your starting location in Southern California, you would have to travel much less to find a Renewable Diesel charging station, compared to a starting location in Northern Canada.

# Explore (3)

Finally, let's look at the actually distance you would have to travel to reach on of these stations. We can create a bar graph to compare the distance traveled for each fuel type. This will just give another to see the relative distance someone would have to travel to reach on of these stations. The distance between two points is only for the shortest path between them, it doesn't account for any actual routes you would take to reach these stations.

In [ ]:
sorted_dist = sorted(distances.items(),key=lambda x:x[1])
sorted_colors = sorted(colors.items(),key=lambda x:x[1])
colors_df = pd.DataFrame(sorted_colors,columns=['Fuel Type','color'])
distances_df = pd.DataFrame(sorted_dist,columns=['Fuel Type','Distance(km)'])
distances_df = distances_df.merge(colors_df)

fig = px.bar(distances_df,x='Fuel Type',y='Distance(km)',color='color',title='Distance to closest fueling station based on fuel type')
fig.update_layout(showlegend=False)
fig.show()

# Communicate

Below are some writing prompts to help you reflect on the new information that is presented from the data. When we look at the evidence, think about what you perceive about the information. Is this perception based on what the evidence shows? If others were to view it, what perceptions might they have?

- I used to think ____________________but now I know____________________. 
- I wish I knew more about ____________________. 
- This visualization reminds me of ____________________. 
- I really like ____________________.

## Add your comments here

✏️

## Saving your work

You can download this notebook, with your comments, by using the **File** menu item in the toolbar above. You might like to downloads this in .html format so that the graphics remain active and can be viewed in your web browser.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)
